In [43]:
plot_type = 'bar' #181 61 12 = 254개 --> 201 72 12 = 285개

In [44]:
import os
file_list=[]
# labeling_group=['도표사진_윤진 라벨링', '도표사진_혜리 라벨링', '도표사진_경진 라벨링']
# labeling_group=['클래스79 라벨링\\'+l for l in labeling_group]
labeling_group=['도표사진_서윤 라벨링', '도표사진_영빈 라벨링', '도표사진_일웅 라벨링', '도표사진_주봉 라벨링', '도표사진_준영 라벨링', '도표사진_현수 라벨링', '도표사진_혜리 라벨링']
labeling_group=['클래스11 라벨링\\'+l for l in labeling_group]
img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\"
for g in labeling_group:
    for l in os.listdir(os.path.join(img_path, g)):
        file_list.append(os.path.join(img_path, g, l))
        
xml_file_list = []
img_file_list = []
for i, j in enumerate(file_list):
    if j.endswith('.xml') and file_list[i-1].endswith(('.png', '.PNG')):
        xml_file_list.append(j)
        img_file_list.append(file_list[i-1])



In [45]:
xml_file_list[-1]

'C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\클래스11 라벨링\\도표사진_혜리 라벨링\\(기본)18-1(4.30)사회주택의 국내외 사례분석과 금융지원 방안… 장한익 김병국)_2.xml'

In [55]:
import pandas as pd
qa_path =  "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\221215 DIAL 정책지원 AI 데이터 관리.xlsx"
qa_data = pd.read_excel(qa_path, sheet_name='데이터 목록 (도표)')
header = qa_data.iloc[4, 1:8]
qa_data = qa_data.iloc[5:, 1:8]
qa_data.columns = header
qa_data = qa_data.reset_index(drop=True)
qa_data.head()

4,보고서 파일,도표 이름,도표 파일 (보고서 파일_인덱스),작성자,질문,답변 (단위는 제외해주세요!),분류
0,‘청년주택’을 넘어 ‘20대주택’으로,〈그림 2〉 2009년~2018년간서울시 구별20~29세 인구 순유입,‘청년주택’을 넘어 ‘20대주택’으로_1,최민규,2009년~2018년간서울시 구별20~29세 인구 순유입이 가장 많은 구는 어디인가?,관악구,막대
1,‘청년주택’을 넘어 ‘20대주택’으로,〈그림 3〉 2009년~2018년간서울시 대학지역20~29세 인구의 시도간유입 추이,‘청년주택’을 넘어 ‘20대주택’으로_2,최민규,2018년 서울시 대학지역20~29세 인구의 시도간유입이 15000명 이하인 지역은...,마포-서대문,꺾은선
2,‘청년주택’을 넘어 ‘20대주택’으로,〈그림 4〉 20~29세 인구의 서울 전입 순이동 추이,‘청년주택’을 넘어 ‘20대주택’으로_3,최민규,25~29세 인구의 서울 전입 순이동이 가장 적었던 년도는 몇년인가?,2015년,꺾은선
3,(2차웨이브) 2020년 장애인고용패널조사(2차웨이브 5차조사),〔그림1-4-2〕조사진행추이,(2차웨이브) 2020년 장애인고용패널조사(2차웨이브 5차조사)_1,최민규,7주차의 5차조사 진행추이는 몇퍼센트 인가?,68.5,꺾은선
4,(21-10호)_월간_ICT산업동향,그림1-1전체반도체시장전망및메모리반도체고정거래가격추이,(21-10호)_월간_ICT산업동향_1,최민규,21.1에서의 D램 DDR4 8GB의 고정거래가격은?,3.00,꺾은선


In [56]:
import cv2
import xmltodict
import json 
import pickle 
import numpy as np

plot_list = []
qid = 1000000
error_cnt = 0 
for index, (img_file, xml_file) in enumerate(zip(img_file_list, xml_file_list)):
    img_file = img_file.split('\\')[-1]
    with open(xml_file, 'rt', encoding='UTF-8') as f:
        xmlString = f.read()
    
    anno_data_list = xmltodict.parse(xmlString)['annotation']['object']
    bboxes = []
    for anno_data in anno_data_list:
        coordinates = list(map(int, list(anno_data['bndbox'].values()))) 
        if (np.array(coordinates) < 0).sum() != 0:
            print("Negative coordinates: ", xml_file.split('\\')[-2:])
        if '\\' in anno_data['name']: 
            print(xml_file.split('\\')[-2:])
            break
        plot_cls = anno_data['name'].rsplit('==', 1)
        # if plot_cls[0] =='digital grade restrictiveness index': print(xml_file.split('\\')[-2:])
        if len(plot_cls) == 2:
            bboxes.append((plot_cls[0], (coordinates), plot_cls[1])) 
        elif len(plot_cls) == 1:
            bboxes.append((plot_cls[0], (coordinates), None))
    try:
        is_line = False
        for bb in bboxes:
            if plot_type in bb[0]: is_line = True
        if is_line:
            plot_list.append(
                {
                    "qid": qid,
                    "question": list(qa_data.loc[qa_data['도표 파일  (보고서 파일_인덱스)'] == img_file[:-4]]['질문'])[0],
                    "answer": list(qa_data.loc[qa_data['도표 파일  (보고서 파일_인덱스)'] == img_file[:-4]]['답변 (단위는 제외해주세요!)'])[0],
                    "image": img_file,
                    "bboxes": bboxes,
                }
            )
            # print(plot_list)
    except:
        error_cnt += 1 
        # print(img_file)
        # print(list(qa_data.loc[qa_data['도표 파일  (보고서 파일_인덱스)'] == img_file[:-4]]['질문']))
        # print(list(qa_data.loc[qa_data['도표 파일  (보고서 파일_인덱스)'] == img_file[:-4]]['답변 (단위는 제외해주세요!)']))
        # from IPython import embed ; embed()
        # print("error")
        continue

    qid += 1
    # if index > 10 : break
print("error", error_cnt)
with open(f'data/h11_{plot_type}_annotation.pkl',"wb") as f:
    pickle.dump(plot_list, f)

9-3-4.PNG
9_4-6.PNG
9_4-7.PNG
(붙임)벤처기업의 혁신과 성장을 위한 벤처자금 생태계 발전 방향_3.png
6_3-24-1.PNG
6_3-24-2.PNG
8_2-5.PNG
7_2-3-1.PNG
error 8


In [41]:

with open(f'data/h11_{plot_type}_annotation.pkl', 'rb') as handle:
    output = pickle.load(handle)
    # print(len(output), output)

In [ ]:
output[0]

In [25]:
import os
from shutil import copyfile

def mkdirs(newdir,mode=777):
    try:
        os.makedirs(newdir, mode)
    except OSError as err:
        return err

file_list=[]
# labeling_group=['도표사진_윤진 라벨링', '도표사진_혜리 라벨링', '도표사진_경진 라벨링']
labeling_group=['도표사진_서윤 라벨링', '도표사진_영빈 라벨링', '도표사진_일웅 라벨링', '도표사진_주봉 라벨링', '도표사진_준영 라벨링', '도표사진_현수 라벨링', '도표사진_혜리 라벨링']
labeling_group=['클래스11 라벨링\\'+l for l in labeling_group]
img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\"

# mkdirs(os.path.join(img_path, 'human11'))

for g in labeling_group:
    for l in os.listdir(os.path.join(img_path, g)):
        file_list.append(os.path.join(img_path, g, l))
        
xml_file_list = []
img_file_list = []
for i, j in enumerate(file_list):
    if j.endswith('.PNG'):
        # print(j)
        copyfile(j , os.path.join(img_path, 'human11', j.split('\\')[-1]))
    if j.endswith('.png'):
        # print(j)
        copyfile(j , os.path.join(img_path, 'human11', j.split('\\')[-1]))



In [ ]:
import os
file_list=[]
labeling_group=['샘플 라벨링']
img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\"
for g in labeling_group:
    for l in os.listdir(os.path.join(img_path, g)):
        file_list.append(os.path.join(img_path, g, l))
xml_file_list = []
img_file_list = []
for i, j in enumerate(file_list):
    if j.endswith('.xml'):
        try:
            j[:-3] == file_list[i-1][:-3]
            xml_file_list.append(j)
            img_file_list.append(file_list[i-1])
        except: print(j, file_list[i-1])



In [ ]:
qa_data = [
    {
        "problem_no": "10",
        "task_no": "3",
        "ref_file": "agc_asdf2hg436tgwdf_figure_10.png",
        "question": "장학 및 컨설팅 경험이 있는 어린이집의 비율은 얼마인가?",
        "answer": "26.8" 
    },
    {
        "problem_no": "34",
        "task_no": "3",
        "ref_file": "agc_asdf2hg436tgwdf_figure_34.png",
        "question": "전통문양 콘텐츠 활용 패션 디자인 분야 중 가장 큰 비중을 차지하는 분야는 무엇인가?",
        "answer": "의류"
    },
    {
        "problem_no": "36",
        "task_no": "3",
        "ref_file": "agc_asdf2hg436tgwdf_figure_36.png",
        "question": "문화포털 전통문양 아카이브 만족도 조사에서 경제성 항목에서 만족이라고 응답한 비율은 얼마인가?",
        "answer": "73.8"
    },
    {
        "problem_no": "39",
        "task_no": "3",
        "ref_file": "AGC_p9aDe43fE2_figure_39.png",
        "question": "문화포털 전통문양 아카이브에 대한 비이용 이유로 콘텐츠 품질과 관련하여 응답한 비율은 얼마인가?",
        "answer": "15.0"
    },
    {
        "problem_no": "40",
        "task_no": "3",
        "ref_file": "AGC_p9aDe43fE2_figure_40.png",
        "question": "2022년 2분기를 기준으로 가격이 가장 낮은 곡물은 무엇인가?",
        "answer": "쌀"
    }
]

In [ ]:
import cv2
import xmltodict
import json 
import pickle 

plot_list = []
qid = 990000
for index, (img_file, xml_file) in enumerate(zip(img_file_list, xml_file_list)):
    img_file = img_file.split('\\')[-1]
    with open(xml_file, 'rt', encoding='UTF-8') as f:
        xmlString = f.read()
    
    anno_data_list = xmltodict.parse(xmlString)['annotation']['object']
    bboxes = []
    
    for anno_data in anno_data_list:
        coordinates = list(map(int, list(anno_data['bndbox'].values()))) 
        if (np.array(coordinates) < 0).sum() != 0:
            print("Negative coordinates: ", xml_file.split('\\')[-2:])
        if '\\' in anno_data['name']: 
            print(xml_file.split('\\')[-2:])
            break
        plot_cls = anno_data['name'].rsplit('==', 1)
        # if plot_cls[0] =='digital grade restrictiveness index': print(xml_file.split('\\')[-2:])
        if len(plot_cls) == 2:
            bboxes.append((plot_cls[0], (coordinates), plot_cls[1]))
        elif len(plot_cls) == 1:
            bboxes.append((plot_cls[0], (coordinates), None))
    try:
        is_line = False
        for bb in bboxes:
            if plot_type in bb[0]: is_line = True
        if is_line:
            for i, j in enumerate(qa_data):
                if j["ref_file"]==img_file:
                    question = qa_data[i]['question']
                    answer = qa_data[i]['answer']
            plot_list.append(
                {
                    "qid": qid,
                    "question": question,
                    "answer": answer,
                    "image": img_file,
                    "bboxes": bboxes,
                }
            )
    except:
        # print("error")
        continue

    qid += 1
    # if index > 10 : break

with open(f'data/s_{plot_type}_annotation.pkl',"wb") as f:
    pickle.dump(plot_list, f)

In [ ]:

with open(f'data/s_{plot_type}_annotation.pkl', 'rb') as handle:
    output = pickle.load(handle)
    # print(len(output), output)

In [ ]:
import os
from shutil import copyfile

def mkdirs(newdir,mode=777):
    try:
        os.makedirs(newdir, mode)
    except OSError as err:
        return err

file_list=[]
labeling_group=['샘플 라벨링']
img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\"

# mkdirs(os.path.join(img_path, 'sample'))

for g in labeling_group:
    for l in os.listdir(os.path.join(img_path, g)):
        file_list.append(os.path.join(img_path, g, l))
        
xml_file_list = []
img_file_list = []
for i, j in enumerate(file_list):
    if j.endswith('.png'):
        # print(j)
        copyfile(j , os.path.join(img_path, 'sample', j.split('\\')[-1]))



In [ ]:
xml_file_list[0]

'(173호)_월간_ICT산업동향+(최종)_2.xml'

In [5]:
import os
import numpy as np

file_list=[]
labeling_group=['도표사진_윤진 라벨링', '도표사진_혜리 라벨링', '도표사진_경진 라벨링']
img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\클래스79 라벨링"
for g in labeling_group:
    for l in os.listdir(os.path.join(img_path, g)):
        file_list.append(os.path.join(img_path, g, l))
        
xml_file_list = []
img_file_list = []
for i, j in enumerate(file_list):
    if j.endswith('.xml'):
        xml_file_list.append(j.split("\\")[-1][:-4])
        img_file_list.append(file_list[i-1])
        try:
            j[:-3] == file_list[i-1][:-3]
        except: print(j, file_list[i-1])


# labeling_group=['제외 도표']
labeling_group=['도표사진_서윤 라벨링', '도표사진_영빈 라벨링', '도표사진_일웅 라벨링', '도표사진_주봉 라벨링', '도표사진_준영 라벨링', '도표사진_현수 라벨링', '도표사진_혜리 라벨링']
img_path = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\클래스11 라벨링"
group_count = []
for g in labeling_group:
    f = open(f"{g}.txt", 'w', encoding="UTF-8")
    cnt = 0 
    for l in os.listdir(os.path.join(img_path, g)):
        if l.endswith('.xml'): cnt+=1
        if l[:-4] in xml_file_list: #이미 라벨링된 이미지 
            # print(l[:-4], xml_file_list[0])
            f.write(l+"\n")
    group_count.append(cnt)
    f.close()
print("서윤", "영빈", "일웅", "주봉", "준영", "현수", "혜리")
print(group_count, np.sum(group_count))
    

서윤 영빈 일웅 주봉 준영 현수 혜리
[53, 100, 31, 44, 22, 61, 3] 314
